In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

last_scrape = '2023-11-27'
# playURL = 'https://api-web.nhle.com/v1/gamecenter/2023020175/play-by-play'
# shiftURL = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=2023020175'

gameDate = '2023-01-09'
awayTeamID = 0
awayTeamName = 'AWAY'
homeTeamID = 0
homeTeamName = 'HOME'


#These are needed for playerIDs
file_name = f"data/summary_statsS-2023-11-28.csv"
player_biosS = pd.read_csv(file_name, index_col=False)
file_name = f"data/summary_statsG-2023-11-28.csv"
player_biosG = pd.read_csv(file_name, index_col=False)
for index, row in player_biosG.iterrows():
    player_biosG.at[index, 'tertiaryPosition'] = 'G'
player_bios = pd.concat([player_biosS, player_biosG], axis=0)
# player_bios

In [2]:
# Gets the basic game data, such as team ID and abbrev

def extract_values(row, key):
    return row.get(key)

def extract_values(row, keys):
    if isinstance(row, dict):
        return {key: row.get(key) for key in keys}
    else:
        return {key: None for key in keys}

def get_game_data(gameID):
    global gameDate, awayTeamID, awayTeamName, homeTeamID, homeTeamName

    playURL = 'https://api-web.nhle.com/v1/gamecenter/' + str(gameID) + '/play-by-play'

    plays = requests.get(playURL).json()


    gameDate = plays['gameDate']
    awayTeamID = plays['awayTeam']['id']
    awayTeamName = plays['awayTeam']['abbrev']
    homeTeamID = plays['homeTeam']['id']
    homeTeamName = plays['homeTeam']['abbrev']
    
    plays = plays['plays']
    
    plays = pd.DataFrame(plays)
    
    for index, row in plays.iterrows():
        if row['typeDescKey'] == 'faceoff':
            plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))+1
        else:
            plays.at[index, 'second'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['timeInPeriod'].split(':'))))

    plays['second'] = plays['second'].astype(int)
    
    
    # These are used to explode the dictionaries within the dataframe and give them their own column
    keys_to_extract1 = ['periodType']

    for key in keys_to_extract1:
        plays[key] = plays['periodDescriptor'].apply(lambda x: extract_values(x, key))

    keys_to_extract_faceoff = ['eventOwnerTeamId', 'losingPlayerId', 'winningPlayerId', 'xCoord', 'yCoord', 'zoneCode']
    keys_to_extract_takeaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
    keys_to_extract_shot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'awaySOG', 'homeSOG']
    keys_to_extract_hit = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'hittingPlayerId', 'hitteePlayerId']
    keys_to_extract_missedshot = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'shootingPlayerId', 'goalieInNetId', 'reason']
    keys_to_extract_block = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode',  'blockingPlayerId', 'shootingPlayerId']
    keys_to_extract_giveaway = ['eventOwnerTeamId', 'playerId', 'xCoord', 'yCoord', 'zoneCode']
    keys_to_extract_stoppage = ['reason']
    keys_to_extract_goal = ['eventOwnerTeamId', 'xCoord', 'yCoord', 'zoneCode', 'shotType', 'scoringPlayerId', 'scoringPlayerTotal', 'assist1PlayerId', 'assist1PlayerTotal', 'assist2PlayerId', 'assist2PlayerTotal', 'goalieInNetId', 'awayScore', 'homeScore']
    keys_to_extract_penalty = ['eventOwnerTeamId', 'committedByPlayerId', 'xCoord', 'yCoord', 'zoneCode', 'typeCode', 'descKey', 'duration', 'drawnByPlayerId']

    keys_lists = [
        keys_to_extract_faceoff,
        keys_to_extract_takeaway,
        keys_to_extract_shot,
        keys_to_extract_hit,
        keys_to_extract_missedshot,
        keys_to_extract_block,
        keys_to_extract_giveaway,
        keys_to_extract_stoppage,
        keys_to_extract_goal,
        keys_to_extract_penalty
    ]
    
    for keys_to_extract in keys_lists:
        # Apply the function to each row
        extracted_df = pd.DataFrame(plays['details'].apply(lambda x: extract_values(x, keys_to_extract)).tolist())
        plays = pd.concat([plays, extracted_df], axis=1)

    plays = plays.groupby(plays.columns, axis=1).first()
    plays = plays.drop(columns=['details', 'periodDescriptor'])
    
    # This is for converting data types
    convert_columns = ['assist1PlayerId', 'assist2PlayerId', 'blockingPlayerId', 'committedByPlayerId', 'drawnByPlayerId', 'goalieInNetId', 'hitteePlayerId', 'hittingPlayerId', 'losingPlayerId', 'playerId', 'scoringPlayerId', 'shootingPlayerId', 'winningPlayerId']

    plays[convert_columns] = plays[convert_columns].astype(pd.Int64Dtype(), errors='ignore')

    convert_columns2 = ['assist1PlayerTotal', 'assist2PlayerTotal', 'awaySOG', 'awayScore', 'homeSOG', 'homeScore', 'scoringPlayerTotal', 'eventOwnerTeamId', 'duration']
    plays[convert_columns2] = plays[convert_columns2].astype(pd.Int64Dtype(), errors='ignore')
    
    # Add one second to faceoff to separate from goals
#     plays.loc[plays['typeDescKey'] == 'faceoff', 'second'] += 1
    
    #Convert to Universal Second by using period
    for i, r in plays.iterrows():
        if r['period'] == 2:
            plays.at[i, 'second'] = r['second'] + 1200
        if r['period'] == 3:
            plays.at[i, 'second'] = r['second'] + 2400
        if r['period'] == 4:
            plays.at[i, 'second'] = r['second'] + 3600

    
    return plays
    


In [3]:
# This queries the NHL api for a shift json, then makes a dataframe and removes the "events"

def get_shift_data(gameID):
    
    shiftURL = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=' + str(gameID)
    
    shifts = requests.get(shiftURL).json()
    shifts = shifts['data']

    shifts_df = pd.DataFrame()

    for shift in shifts:
        temp = pd.DataFrame([shift])
        shifts_df = pd.concat([shifts_df, temp], axis=0, ignore_index=True)
        
    #Separate the events    
    events = shifts_df.loc[~shifts_df['eventDescription'].isna()]
    shifts_df = shifts_df.loc[shifts_df['eventDescription'].isna()]  
    
    #Add Player IDs
    for index, row in shifts_df.iterrows():
        pos = player_bios.loc[player_bios['playerId'] == row['playerId']]['tertiaryPosition'].iloc[0]
        shifts_df.at[index, 'position'] = pos
        
    #Add a row with the second as an integer
    for index, row in shifts_df.iterrows():
        shifts_df.at[index, 'newStart'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['startTime'].split(':'))))
        shifts_df.at[index, 'newEnd'] = sum(int(x) * 60**i for i, x in enumerate(reversed(row['endTime'].split(':'))))
    
    shifts_df['newStart'] = shifts_df['newStart'].astype(int)
    shifts_df['newEnd'] = shifts_df['newEnd'].astype(int)
    
    #Convert to Universal Second by using period
    for index, row in shifts_df.iterrows():
        if row['period'] == 2:
            shifts_df.at[index, 'newStart'] = row['newStart'] + 1200
            shifts_df.at[index, 'newEnd'] = row['newEnd'] + 1200
        if row['period'] == 3:
            shifts_df.at[index, 'newStart'] = row['newStart'] + 2400
            shifts_df.at[index, 'newEnd'] = row['newEnd'] + 2400
        if row['period'] == 4:
            shifts_df.at[index, 'newStart'] = row['newStart'] + 3600
            shifts_df.at[index, 'newEnd'] = row['newEnd'] + 3600

    return shifts_df 



In [4]:
# This returns the line table if given a df of only one team and one position type

def line_table_team_pos(df):
    onice = pd.DataFrame()

    for i in range(4800):
        t = i+1
        activeName = []
        activeId = []
        for index, row in df.iterrows():
            if (t > row['newStart']) & (t <=row['newEnd']):
                activeName.append(f"{row['firstName']} {row['lastName']}")
                activeId.append(row['playerId'])
        activeName += [''] * (6 - len(activeName))
        activeId += [''] * (6 - len(activeId))
        temp = {f'p{k+1}': activeName[k] for k in range(6)}
        temp2 = {f'ID{k+1}': activeId[k] for k in range(6)}
        temp.update(temp2)
        temp['second'] = t

        temp['line'] = (temp['p1'] + '+' + temp['p2'] + '+' + temp['p3'] + '+' + temp['p4'] + '+' + temp['p5'] + '+' + temp['p6']).rstrip('+')
        temp['lineId'] = (str(temp['ID1']) + '+' + str(temp['ID2']) + '+' + str(temp['ID3']) + '+' + str(temp['ID4']) + '+' + str(temp['ID5']) + '+' + str(temp['ID6'])).rstrip('+')
        temp = pd.DataFrame([temp])

        onice = pd.concat([onice, temp], axis=0, ignore_index=True)
        

                          
    return onice

In [5]:


def pp_shifts(df):
    
    pp_df = pd.DataFrame()

    for i in range(4800):
        t = i+1
        activeName = []
        activeId = []
        activeTeam = []
        for index, row in df.iterrows():
            if (t > row['newStart']) & (t <=row['newEnd']):
                activeId.append(row['playerId'])
                activeTeam.append(row['teamAbbrev'])

        activeId += [''] * (12 - len(activeId))
        activeTeam += [''] * (12 - len(activeTeam))
        temp = {f'PP{k+1}': activeTeam[k] for k in range(12)}
        temp['second'] = t
    
        temp = pd.DataFrame([temp])

        pp_df = pd.concat([pp_df, temp], axis=0, ignore_index=True)
        
    for index, row in pp_df.iterrows():
        away = 0
        home = 0
        for k in range(12):
            p = k+1
            if row[f"PP{p}"] == homeTeamName:
                home +=1
            elif row[f"PP{p}"] == awayTeamName:
                away +=1
        pp_df.at[index, 'home_players'] = home
        pp_df.at[index, 'away_players'] = away

        if home > away:
            pp_df.at[index, 'home_pp'] = 1
            pp_df.at[index, 'away_pp'] = 0
        elif home < away:
            pp_df.at[index, 'home_pp'] = 0
            pp_df.at[index, 'away_pp'] = 1
        else:
            pp_df.at[index, 'home_pp'] = 0
            pp_df.at[index, 'away_pp'] = 0
            
    pp_df['home_players'] = pp_df['home_players'].astype(int)
    pp_df['home_pp'] = pp_df['home_pp'].astype(int)
    pp_df['away_players'] = pp_df['away_players'].astype(int)
    pp_df['away_pp'] = pp_df['away_pp'].astype(int)
        
    return pp_df

In [6]:

def goalie_present(df):
    
    for index, row in df.iterrows():
        if row['line'] == '':
            df.at[index, 'isGoalie'] = 0
        else:
            df.at[index, 'isGoalie'] = 1
            
    df['isGoalie'] = df['isGoalie'].astype(int)
    
    df = df[['second', 'isGoalie']]
            
    return df

In [7]:
game_data = get_game_data(2023020462)
shift_data = get_shift_data(2023020462)

ppfinder = shift_data.copy()
pp_boolean = pp_shifts(ppfinder)

In [8]:
awayF = shift_data.loc[(shift_data['teamAbbrev'] == awayTeamName) & (shift_data['position'] == 'F')]
awayD = shift_data.loc[(shift_data['teamAbbrev'] == awayTeamName) & (shift_data['position'] == 'D')]
awayG = shift_data.loc[(shift_data['teamAbbrev'] == awayTeamName) & (shift_data['position'] == 'G')]

homeF = shift_data.loc[(shift_data['teamAbbrev'] == homeTeamName) & (shift_data['position'] == 'F')]
homeD = shift_data.loc[(shift_data['teamAbbrev'] == homeTeamName) & (shift_data['position'] == 'D')]
homeG = shift_data.loc[(shift_data['teamAbbrev'] == homeTeamName) & (shift_data['position'] == 'G')]

awayF

,id,detailCode,duration,endTime,eventDescription,eventDetails,eventNumber,firstName,gameId,hexValue,...,playerId,shiftNumber,startTime,teamAbbrev,teamId,teamName,typeCode,position,newStart,newEnd
0,13595869,0,00:33,00:39,None,None,55,Blake,2023020462,#0033A0,...,8471218,1,00:06,NYR,3,New York Rangers,517,F,6,39
1,13595870,0,00:33,03:35,None,None,85,Blake,2023020462,#0033A0,...,8471218,2,03:02,NYR,3,New York Rangers,517,F,182,215
2,13596312,0,00:45,07:13,None,None,224,Blake,2023020462,#0033A0,...,8471218,3,06:28,NYR,3,New York Rangers,517,F,388,433
3,13596313,0,00:45,10:49,None,None,314,Blake,2023020462,#0033A0,...,8471218,4,10:04,NYR,3,New York Rangers,517,F,604,649
4,13596877,0,00:31,14:30,None,None,402,Blake,2023020462,#0033A0,...,8471218,5,13:59,NYR,3,New York Rangers,517,F,839,870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,13597853,0,00:23,04:30,None,None,626,Will,2023020462,#0033A0,...,8482157,6,04:07,NYR,3,New York Rangers,517,F,1447,1470
698,13597854,0,00:34,08:39,None,None,665,Will,2023020462,#0033A0,...,8482157,7,08:05,NYR,3,New York Rangers,517,F,1685,1719
699,13597855,0,00:33,09:55,None,None,678,Will,2023020462,#0033A0,...,8482157,8,09:22,NYR,3,New York Rangers,517,F,1762,1795
700,13599069,0,00:39,17:46,None,None,814,Will,2023020462,#0033A0,...,8482157,9,17:07,NYR,3,New York Rangers,517,F,2227,2266


In [9]:
homeLinesF = line_table_team_pos(homeF)
homeLinesD = line_table_team_pos(homeD)

awayLinesF = line_table_team_pos(awayF)
awayLinesD = line_table_team_pos(awayD)

homeGoalieBoolean = goalie_present(line_table_team_pos(homeG))
homeGoalieBoolean.rename(columns={'isGoalie': 'isGoalieHome'}, inplace=True)
awayGoalieBoolean = goalie_present(line_table_team_pos(awayG))
awayGoalieBoolean.rename(columns={'isGoalie': 'isGoalieAway'}, inplace=True)

goalieBoolean = pd.merge(homeGoalieBoolean, awayGoalieBoolean, how='left', on='second')

In [10]:
test = pd.merge(homeLinesF, game_data, how='left', on='second')
test2 = pd.merge(test, pp_boolean, how='left', on='second')
test3 = pd.merge(test2, goalieBoolean, how='left', on='second')

test3.to_excel('testLine.xlsx')

# Testing code below

-----------------

In [22]:
test4 = test3[test3['lineId'].str.contains('8473419')]
test4

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,PP9,PP10,PP11,PP12,home_players,away_players,home_pp,away_pp,isGoalieHome,isGoalieAway
171,Brad Marchand,James van Riemsdyk,Trent Frederic,,,,8473419,8474037,8479365,,...,NYR,NYR,BOS,BOS,6,6,0,0,1,1
172,Brad Marchand,James van Riemsdyk,Trent Frederic,,,,8473419,8474037,8479365,,...,NYR,NYR,BOS,BOS,6,6,0,0,1,1
173,Brad Marchand,Charlie Coyle,Trent Frederic,,,,8473419,8475745,8479365,,...,NYR,BOS,BOS,NYR,6,6,0,0,1,1
174,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,NYR,6,6,0,0,1,1
175,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,NYR,6,6,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3673,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3674,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3675,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1


In [23]:
# Get rid of duped seconds for an accurate count of ice time

mask = ~test4['second'].duplicated()
result = test4[mask]
result

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,PP9,PP10,PP11,PP12,home_players,away_players,home_pp,away_pp,isGoalieHome,isGoalieAway
171,Brad Marchand,James van Riemsdyk,Trent Frederic,,,,8473419,8474037,8479365,,...,NYR,NYR,BOS,BOS,6,6,0,0,1,1
172,Brad Marchand,James van Riemsdyk,Trent Frederic,,,,8473419,8474037,8479365,,...,NYR,NYR,BOS,BOS,6,6,0,0,1,1
173,Brad Marchand,Charlie Coyle,Trent Frederic,,,,8473419,8475745,8479365,,...,NYR,BOS,BOS,NYR,6,6,0,0,1,1
174,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,NYR,6,6,0,0,1,1
175,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,NYR,6,6,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3673,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3674,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1
3675,Brad Marchand,Matthew Poitras,,,,,8473419,8483505,,,...,,,,,4,4,0,0,1,1


In [24]:
# test5 = test4[~test4['typeDescKey'].isna()]

#Get Corsi relevant moments
valid_types = ['shot-on-goal', 'blocked-shot', 'missed-shot', 'goal']
test5 = test4[test4['typeDescKey'].isin(valid_types)]
test5

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,PP9,PP10,PP11,PP12,home_players,away_players,home_pp,away_pp,isGoalieHome,isGoalieAway
179,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,BOS,NYR,BOS,NYR,6,6,0,0,1,1
330,Brad Marchand,David Pastrnak,Morgan Geekie,,,,8473419,8477956,8479987,,...,NYR,BOS,BOS,BOS,6,6,0,0,1,1
581,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,BOS,6,6,0,0,1,1
735,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,BOS,BOS,NYR,NYR,6,6,0,0,1,1
759,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,BOS,BOS,NYR,NYR,6,6,0,0,1,1
972,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,NYR,NYR,BOS,NYR,6,6,0,0,1,0
1010,Brad Marchand,Charlie Coyle,,,,,8473419,8475745,,,...,NYR,NYR,BOS,,5,6,0,1,1,1
1013,Brad Marchand,Charlie Coyle,,,,,8473419,8475745,,,...,NYR,NYR,BOS,,5,6,0,1,1,1
1169,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1178,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1


In [25]:
test6 = test5.loc[test5['typeDescKey'] == 'blocked-shot']
test6
test7 = test5.loc[test5['typeDescKey'] != 'blocked-shot']
test7

,p1,p2,p3,p4,p5,p6,ID1,ID2,ID3,ID4,...,PP9,PP10,PP11,PP12,home_players,away_players,home_pp,away_pp,isGoalieHome,isGoalieAway
330,Brad Marchand,David Pastrnak,Morgan Geekie,,,,8473419,8477956,8479987,,...,NYR,BOS,BOS,BOS,6,6,0,0,1,1
735,Brad Marchand,Charlie Coyle,Jake DeBrusk,,,,8473419,8475745,8478498,,...,BOS,BOS,NYR,NYR,6,6,0,0,1,1
1010,Brad Marchand,Charlie Coyle,,,,,8473419,8475745,,,...,NYR,NYR,BOS,,5,6,0,1,1,1
1013,Brad Marchand,Charlie Coyle,,,,,8473419,8475745,,,...,NYR,NYR,BOS,,5,6,0,1,1,1
1169,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1178,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1242,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1249,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1348,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,NYR,BOS,BOS,,6,5,1,0,1,1
1394,Brad Marchand,James van Riemsdyk,David Pastrnak,Morgan Geekie,,,8473419,8474037,8477956,8479987,...,BOS,BOS,NYR,,6,5,1,0,1,1


In [26]:
test7['eventOwnerTeamId'].value_counts()

6    16
3    15
Name: eventOwnerTeamId, dtype: Int64

In [27]:
test6['eventOwnerTeamId'].value_counts()

3    10
6     5
Name: eventOwnerTeamId, dtype: Int64

In [28]:
homeTeamID

6

In [18]:
# test8 = test7.loc[(test7['home_players'] == 6) & (test7['away_players'] == 6)]
# test8['eventOwnerTeamId'].value_counts()

In [19]:
# test9 = test6.loc[(test6['home_players'] == 6) & (test6['away_players'] == 6)]
# test9['eventOwnerTeamId'].value_counts()

In [20]:
# test6.to_excel('testingblocks.xlsx')

In [21]:
# homeF.loc[(homeF['period'] == 2) & (homeF['playerId'] == 8474641)]

In [50]:
playURL = 'https://api-web.nhle.com/v1/gamecenter/' + str(2023020462) + '/play-by-play'
test = requests.get(playURL).json()
test['rosterSpots']

jerseys = pd.DataFrame()

for spot in test['rosterSpots']:
    teamId = spot.get('teamId')
    sweaterNumber = spot.get('sweaterNumber')
    playerId = spot.get('playerId')
    
    temp = pd.DataFrame([{'teamId': teamId, 'sweaterNumber': sweaterNumber, 'playerId': playerId}])
    
    jerseys = pd.concat([jerseys, temp], ignore_index=True)

jerseys

,teamId,sweaterNumber,playerId
0,3,17,8471218
1,3,32,8471734
2,6,63,8473419
3,3,12,8474009
4,6,12,8474031
5,6,21,8474037
6,3,20,8475184
7,6,13,8475745
8,3,71,8475752
9,3,16,8476389


40